# Car example in Reinforcement Learning
---
Can a computer learn to drive a virtual racecar down a racetrack?

In [30]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

In [4]:
# code from Gymnasium webpage: https://gymnasium.farama.org/
# env = gym.make("CarRacing-v2", render_mode="human")
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#    action = env.action_space.sample()  # this is where you would insert your policy
#    observation, reward, terminated, truncated, info = env.step(action)

#    if terminated or truncated:
#       observation, info = env.reset()

# env.close()

## Problem setup
---
Write here about the agent, environment, rewards, etc.

Leveraging a 'top down' approach for agent observations (I think)

In [18]:
environment_name = "CarRacing-v2"
env = gym.make(environment_name, render_mode='human')

In [19]:
# track generation
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {})

In [20]:
env.action_space

Box([-1.  0.  0.], 1.0, (3,), float32)

In [21]:
# racing track, 96 x 96 image with 3 colour overlays
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [22]:
# produces new window with racetrack environment
env.render()

In [23]:
# close opened track environment
env.close()

You can observe the car's path when taking random actions by running the below code. An episode will terminate when either the car visits all the track tiles, or it falls off the racetrack (in which case it receives reward -100).

In [24]:
# episodes = 5
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
    
#     while not done:
#         env.render()
#         # random action
#         action = env.action_space.sample()
#         n_state, reward, done, info, _ = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))
# env.close()

## Model training
---
We will train our racecar using the Proximal Policy Optimisation (PPO) algorithm.

In [27]:
log_path = os.path.join('Training', 'Logs')

In [28]:
log_path

'Training\\Logs'

The `CnnPolicy` is able to deal with image recognition, which is how our agent observes the problem.

In [31]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
# multi-layer perceptron policy
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [32]:
# train the model
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 57   |
|    iterations      | 1    |
|    time_elapsed    | 35   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 46          |
|    iterations           | 2           |
|    time_elapsed         | 88          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005282553 |
|    clip_fraction        | 0.0417      |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.26       |
|    explained_variance   | 0.00393     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.287       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00446    |
|    std                  | 1           |
|    value_loss           | 0.616       |
---

## Saving the model

In [33]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_racecar')

In [34]:
# save model to specified location
model.save(PPO_Path)

In [21]:
# delete and reload model
# del model
# model = PPO.load(PPO_Path, env=env)

## Model evaluation

In [14]:
# from stable_baselines3.common.evaluation import evaluate_policy

In [40]:
environment_name = "CarRacing-v2"
env = gym.make(environment_name, render_mode='human')

In [41]:
evaluate_policy(model, env, n_eval_episodes=10)

KeyboardInterrupt: 

In [38]:
env.close()

## Model testing

In [28]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        # actions dictated by the trained model
        action, _ = model.predict(obs)
        n_state, reward, done, info, = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[17.]
Episode:2 Score:[20.]
Episode:3 Score:[11.]
Episode:4 Score:[17.]
Episode:5 Score:[9.]


In [30]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [32]:
!tensorboard --logdir={training_log_path}

^C
